# Introduction

In this assignment you will be given a series of tasks about using the library `power-grid-model`. The tasks include:

1. [Load input](#Assignment-1:-Load-Input-Data)
2. [Validate Input Data](#Assignment-2:-Validate-Input-Data)
3. [Construct Model](#Assignment-3:-Construct-Model)
4. [Calculate One Time Power Flow](#Assignment-4:-Calculate-One-Time-Power-Flow)
5. [Time Series Batch Calculation](#Assignment-5:-Time-Series-Batch-Calculation)
6. [N 1 Scenario-Batch-Calculation](#Assignment-6:-N-1-Scenario-Batch-Calculation)

The input data are CSV files in the `data/` folder:
* `node.csv`
* `line.csv`
* `source.csv`
* `sym_load.csv`


# Preparation

First import everything we need for this workshop:

In [1]:
import time
from typing import Dict

import numpy as np
import pandas as pd

from power_grid_model import (
    PowerGridModel,
    CalculationType,
    CalculationMethod,
    initialize_array
)

from power_grid_model.validation import (
    assert_valid_input_data,
    assert_valid_batch_data
)

Let's define a timer class to easily benchmark the calculations:

In [2]:
class Timer:
    def __init__(self, name: str):
        self.name = name
        self.start = None

    def __enter__(self):
        self.start = time.perf_counter()

    def __exit__(self, *args):
        print(f'Execution time for {self.name} is {(time.perf_counter() - self.start):0.6f} s')

The following example measures the time for a simple add operation of two numpy arrays.

In [3]:
a = np.random.rand(1000000)
b = np.random.rand(1000000)
with Timer("Add Operation"):
    c = a + b

Execution time for Add Operation is 0.002384 s


# Assignment 1: Load Input Data

The following function loads the CSV data files from folder `data/` and convert them into one dictionary of numpy structured arrays. The returned dictionary is a compatible input for the constructor of `PowerGridModel`. Please complete the function to construct the input data which is compatible with `PowerGridModel`.

In [4]:
def load_input_data() -> Dict[str, np.ndarray]:
    input_data = {}
    for component in ['node', 'line', 'source', 'sym_load']:
        
        # Use pandas to read CSV data
        df = pd.read_csv(f'data/{component}.csv')

        # Initialize array
        input_data[component] = initialize_array('input', component, len(df))

        # Fill the attributes
        for attr, values in df.items():
            input_data[component][attr] = values

        # Print some debug info
        print(f"{component:9s}: {len(input_data[component]):4d}")

    return input_data

# Load input data
with Timer("Loading Input Data"):
    input_data = load_input_data()


node     : 2001
line     : 2000
source   :    1
sym_load : 2000
Execution time for Loading Input Data is 0.066937 s


# Assignment 2: Validate Input Data

It is recommended to validate your data before constructing the `PowerGridModel`. If you are confident about your input data, you can skip this step for performance reasons. The easiest way to validate your input data is using `assert_valid_input_data`, which will raise an exception if there are any errors in your data. Please have a look at the [Validation Examples](https://github.com/alliander-opensource/power-grid-model/blob/main/examples/Validation%20Examples.ipynb) for more detailed information on the validation functions.

In [5]:
# Validate input data
with Timer("Validating Input Data"):
    assert_valid_input_data(input_data=input_data, calculation_type=CalculationType.power_flow)

Execution time for Validating Input Data is 0.041737 s


# Assignment 3: Construct Model

Create an instance of `PowerGridModel` using the input data. Benchmark the construction time.

In [6]:
# Construct model
with Timer("Model Construction"):
    model = PowerGridModel(input_data=input_data)

# Print the number of objects
print(model.all_component_count)

Execution time for Model Construction is 0.001090 s
{'line': 2000, 'node': 2001, 'source': 1, 'sym_load': 2000}


# Assignment 4: Calculate One-Time Power Flow

* Calculate one-time power flow, print the highest and lowest loading of the lines.
* Try with Newton-Raphson and linear method, compare the results and speed.

In [7]:
# Newton-Raphson Power Flow
with Timer("Newton-Raphson Power Flow"):
    result = model.calculate_power_flow(calculation_method=CalculationMethod.newton_raphson)
    
# Print min and max line loading
print("Min line loading:", min(result["line"]["loading"]))
print("Min line loading:", max(result["line"]["loading"]))

Execution time for Newton-Raphson Power Flow is 0.040440 s
Min line loading: 0.14188449783808293
Min line loading: 1.6292378285645628


In [8]:
# Linear Power Flow
with Timer("Linear Power Flow"):
    result = model.calculate_power_flow(calculation_method=CalculationMethod.linear)
    
# Print min and max line loading
print("Min line loading:", min(result["line"]["loading"]))
print("Min line loading:", max(result["line"]["loading"]))

Execution time for Linear Power Flow is 0.006450 s
Min line loading: 0.139568608739431
Min line loading: 1.615684999105591


# Assignment 5: Time Series Batch Calculation

## Load Profile

Below we randomly generate a dataframe of load profile. 

* The column names are the IDs of `sym_load`
* Each row is one scenario
* Each entry specifies the active power of the load
* The reactive power is zero


In [9]:
# Generate random load profile
n_scenarios = 1000
n_loads = len(input_data["sym_load"]) 
load_id = input_data["sym_load"]["id"]
load_p = input_data["sym_load"]["p_specified"]
profile = np.tile(load_p, (n_scenarios, 1)) + 1e4 * np.random.randn(n_scenarios, n_loads)
df_load_profile = pd.DataFrame(profile, columns=load_id)
display(df_load_profile)

,4002,4003,4004,4005,4006,4007,4008,4009,4010,4011,...,5992,5993,5994,5995,5996,5997,5998,5999,6000,6001
0,973663.761608,1.036957e+06,918722.026480,961373.809346,1.085991e+06,912335.024760,926331.816185,1.053110e+06,1.021199e+06,9.979372e+05,...,1.024111e+06,1.068598e+06,1.049798e+06,921451.764663,911085.124714,1.004976e+06,926257.219623,1.089450e+06,916968.277728,1.090162e+06
1,982601.000707,1.048235e+06,905433.487418,958876.266726,1.074536e+06,911733.595706,898931.694346,1.074097e+06,1.030222e+06,1.016769e+06,...,1.013339e+06,1.075035e+06,1.036528e+06,931102.990533,908551.477181,1.000989e+06,939939.992740,1.083321e+06,930118.128285,1.091653e+06
2,994868.532081,1.045230e+06,919337.193418,966581.327541,1.046401e+06,914530.488574,935816.604049,1.046309e+06,1.036578e+06,1.011976e+06,...,1.001730e+06,1.070344e+06,1.039950e+06,928363.708577,908071.524111,1.005031e+06,918775.375956,1.100859e+06,902795.151256,1.081177e+06
3,998051.239505,1.027702e+06,908744.617261,976600.604457,1.075782e+06,897669.629692,936684.239758,1.043375e+06,1.038113e+06,1.023572e+06,...,1.007061e+06,1.071882e+06,1.055484e+06,931572.598632,923954.437280,9.794893e+05,950100.878591,1.092467e+06,923196.213987,1.090656e+06
4,987454.301469,1.057564e+06,920227.354691,939973.645156,1.067011e+06,902923.240212,917788.105205,1.058004e+06,1.017137e+06,1.022795e+06,...,1.004874e+06,1.083570e+06,1.040108e+06,938336.197697,911498.353387,1.002151e+06,946070.309180,1.090226e+06,921013.656178,1.093836e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996727.023806,1.047784e+06,898350.376138,968671.926319,1.077819e+06,910585.792563,928810.506601,1.039977e+06,1.011353e+06,1.008175e+06,...,1.009898e+06,1.086263e+06,1.051729e+06,921357.843993,907554.120793,9.808074e+05,943179.922051,1.103385e+06,918073.568012,1.080327e+06
996,988251.536873,1.050602e+06,885440.819507,979519.537264,1.060508e+06,919616.434080,935674.074205,1.054276e+06,1.018813e+06,1.015506e+06,...,9.973340e+05,1.088739e+06,1.063336e+06,927956.785927,901112.168907,9.690406e+05,924742.052283,1.092355e+06,930222.266595,1.084800e+06
997,998699.359383,1.049457e+06,917090.661485,951736.443689,1.073886e+06,922637.474416,937238.740970,1.069308e+06,1.018909e+06,1.017386e+06,...,1.014314e+06,1.068052e+06,1.050853e+06,930612.443293,888314.789464,1.000656e+06,938865.068272,1.078279e+06,921486.876808,1.089675e+06
998,993861.336844,1.044681e+06,921975.453849,960682.380932,1.051420e+06,926741.994390,944011.368250,1.066315e+06,1.016030e+06,1.005865e+06,...,1.009420e+06,1.083018e+06,1.047633e+06,927514.077989,918385.119663,9.783549e+05,934008.356460,1.074669e+06,915623.231989,1.107427e+06


## Run Time Series Calculation

We want to run a time-series load flow batch calculation using the dataframe.

* Convert the load profile into the compatible batch update dataset.
* Run the batch calculation.
* Compare the calculation methods `newton_raphson` and `linear`.

In [10]:
# Initialize an empty load profile
load_profile = initialize_array("update", "sym_load", df_load_profile.shape)

# Set the attributes for the batch calculation (assume q_specified = 0.0)
load_profile["id"] = df_load_profile.columns.to_numpy()
load_profile["p_specified"] = df_load_profile.to_numpy()
load_profile["q_specified"] = 0.0

# Construct the update data
update_data = {"sym_load": load_profile}

In [11]:
# Validating batch data can take a long time.
# It is recommended to only validate batch data when you run into trouble.
with Timer("Validating Batch Data"):
    assert_valid_batch_data(input_data=input_data, update_data=update_data, calculation_type=CalculationType.power_flow)

Execution time for Validating Batch Data is 31.333834 s


In [12]:
# Run Newton Raphson power flow (this may take a minute...)
with Timer("Batch Calculation using Newton-Raphson"):
    model.calculate_power_flow(update_data=update_data, calculation_method=CalculationMethod.newton_raphson)

Execution time for Batch Calculation using Newton-Raphson is 17.497064 s


In [13]:
# Run linear power flow
with Timer("Batch Calculation using linear calculation"):
    model.calculate_power_flow(update_data=update_data, calculation_method=CalculationMethod.linear)

Execution time for Batch Calculation using linear calculation is 0.965608 s


# Assignment 6: N-1 Scenario Batch Calculation

We want to run a N-1 Scenario analysis. For each batch calculation, one `line` is disconnected at from- and to-side.

In [14]:
n_lines = len(input_data["line"])

# Initialize an empty line profile
line_profile = initialize_array("update", "line", (n_lines, n_lines))

# Set the attributes for the batch calculation
line_profile["id"] =  input_data["line"]["id"]
line_profile["from_status"] = 1 - np.eye(n_lines, dtype=np.uint8)
line_profile["to_status"] = 1 - np.eye(n_lines, dtype=np.uint8)

# Construct the update data
update_data = {"line": line_profile}

In [15]:
# Validating batch data can take a long time.
# It is recommended to only validate batch data when you run into trouble.
with Timer("Validating Batch Data"):
    assert_valid_batch_data(input_data=input_data, update_data=update_data, calculation_type=CalculationType.power_flow)

Execution time for Validating Batch Data is 63.936292 s


In [16]:
# Run Newton Raphson power flow (this may take a minute...)
with Timer("Batch Calculation using Newton-Raphson"):
    model.calculate_power_flow(update_data=update_data, calculation_method=CalculationMethod.newton_raphson)

Execution time for Batch Calculation using Newton-Raphson is 30.987936 s


In [17]:
# Run linear power flow
with Timer("Batch Calculation using linear calculation"):
    model.calculate_power_flow(update_data=update_data, calculation_method=CalculationMethod.linear)

Execution time for Batch Calculation using linear calculation is 3.392957 s


## Parallel processing
The `calculate_power_flow` method has an optional `threading` argument to define the number of threads ran in parallel. Experiment with different threading values and compare the results...

In [18]:
# By default, sequential threading is used
with Timer("Sequential"):
    model.calculate_power_flow(update_data=update_data)

# Single thread, this is essentially the same as running a single thread
with Timer("Single thread"):
    model.calculate_power_flow(update_data=update_data, threading=1)

# Two threads should be faster    
with Timer("Two threads in parallel"):
    model.calculate_power_flow(update_data=update_data, threading=2)

# Four threads should be even faster    
with Timer("Four threads in parallel"):
    model.calculate_power_flow(update_data=update_data, threading=4)

# Use number of threads based the machine hardware    
with Timer("Use number of threads based the machine hardware"):
    model.calculate_power_flow(update_data=update_data, threading=0)

Execution time for Sequential is 29.697149 s
Execution time for Single thread is 30.063583 s
Execution time for Two threads in parallel is 15.683002 s
Execution time for Four threads in parallel is 9.725921 s
Execution time for Recommended: Use number of threads based the machine hardware is 8.020224 s
